In [1]:
# The Code written by Ali Babolhaveji @ 6/7/2020
import sys
import os

package_path = '../'
if not package_path in sys.path:
    sys.path.append(package_path)

import shutil, errno
import yaml

from lib import  ClogLossDataset_from_compressed_data
from torch.utils.data import DataLoader
from lib import  my_deep_clag_loss
from lib import  train_val


import torch
import torch.nn as nn
import gc

with open ('../script/config.yml', 'r') as file:
    cfg = yaml.safe_load(file)
    
train_dataset = ClogLossDataset_from_compressed_data(cfg , split='train')
val_dataset   = ClogLossDataset_from_compressed_data(cfg , split='val')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# ! pip install future

In [3]:
# from lib import create_new_experiment

# this_expr = create_new_experiment(cfg)


In [5]:
len(train_dataset) , len(val_dataset)

(9000, 1000)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=cfg['train']['trainBatchSize'],
                          num_workers=cfg['train']['num_Workers'], shuffle=True , pin_memory=True)

val_loader = DataLoader(val_dataset, batch_size=cfg['train']['valBatchSize'], num_workers=cfg['train']['num_Workers'],
                        shuffle=True ,pin_memory=True)

In [ ]:
img_tensor , meta = next(iter(train_loader))

img_tensor = img_tensor.to(device)
img_tensor.shape

In [ ]:
model = my_deep_clag_loss()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
#     inputs = torch.randn((2, 3, 200, 150, 150)).to(device)
output = model(img_tensor)
print(output.shape)

In [ ]:
criterion = nn.BCELoss()

In [ ]:

rows = range(len(meta['stalled']))
oneHotLabel = torch.zeros(len(meta['stalled']),2)
oneHotLabel[rows ,meta['stalled'].numpy()==1] =1
oneHotLabel ,(meta['stalled'])

oneHotLabel = oneHotLabel.to(device)

criterion(output , oneHotLabel)

In [ ]:
if cfg['model']['load']['flag']:
    state_dict = torch.load(cfg['model']['load']['path'])
    model.load_state_dict(state_dict)
    print(f"[info] Model is loaded. [from {cfg['model']['load']['path']}]")
    Logger(f"Load check point: {cfg['model']['load']['path']} " ,logger)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=cfg['train']['lr']['init'], weight_decay=1e-5)
# criterion = nn.MSELoss()


In [ ]:
for epoch in range(cfg['train']['startEpoch'], cfg['train']['endEpoch']):  # loop over the dataset multiple times
    print(f"=======================  Epoch {epoch} / {cfg['train']['endEpoch']}  =======================")
    gc.collect()



    _, history=train_val(model, 
                         train_loader, 
                         epoch, device, 
                         optimizer, 
                         criterion, 
                         cfg, mode='train')
    
    
    _, history=train_val(model, 
                         val_loader, 
                         epoch, device, 
                         optimizer, 
                         criterion, 
                         cfg, mode='val')



#     # curr_val_loss, history = train_val(model,
#     curr_val_loss, history =train_val(model,
#                                        val_loader,
#                                        criterion=criterion,
#                                        epoch=epoch,
#                                        device=device,
#                                        optimizer=optimizer,
#                                        writer=writer,
#                                        history=history,
#                                        config=cfg,
#                                        logger=logger,
#                                        mode='val')

#     with open(os.path.join(source_save_dir, f'history.pkl'), 'wb') as handle:
#         pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     #

#     #
#     # for name, param in model.named_parameters():
#     #     writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
#     # writer.close()
#     print('curr_val_loss', curr_val_loss)
#     Logger(f"curr_val_loss : {curr_val_loss}" ,logger)
#     #

#     if curr_val_loss < best_loss:
#         model_save_format = f"model_E{epoch:03d}_Loss{curr_val_loss:.6f}.pt"
#         # torch.save(model.state_dict(), os.path.join(experiment_name ,f"./model_E{epoch:03d}_Loss{curr_val_loss:.6f}.pt"))
#         torch.save(model.state_dict(), os.path.join(model_save_dir, model_save_format))
#         # print (f"./model_E{epoch:03d}_Loss{curr_val_loss:.6f}.pt  is saved.")
#         print(os.path.join(model_save_dir, model_save_format) + " is saved.")
#         Logger(f"{os.path.join(model_save_dir, model_save_format)} is saved." ,logger)
#         best_loss = curr_val_loss
#         with open(os.path.join(model_save_dir, f'history.pkl'), 'wb') as handle:
#             pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)